In [119]:
import numpy as np
import math
import sympy as sp


def diff(f, var = 'x y', d = 'y'):
    d = sp.symbols(d)
    x, y = sp.symbols(var)
    
    f_sym = f(x, y)
    
    df = sp.diff(f_sym, d)
    
    df = sp.lambdify((x, y), df)
    return df

dy = lambda t, y: y - t**2 + 1  

a, b = 0, 2
y0 = 0.5


**Método de Euler**

In [120]:
def euler(f, a, b, y0, n = 10):
    h = (b - a) / n
    t = a
    w = y0
    
    for i in range(1, n+1):
        w = w + h * f(t, w)
        t = a + i * h
    return w

w = euler(dy, a, b, y0)
print(w)

4.865784504320001


**Método de Taylor de Ordem Superior**

In [121]:
def taylor(d, a, b, y0, n = 10):
    h = (b - a) / n
    X = np.arange(a, b, h)
    Y = [y0]
    
    order = len(d) + 1
    for j in range(len(X) - 1):
        taylor_sum = Y[j]
        for i in range(1,order):
            taylor_sum += (d[i-1](X[j], Y[j]) * h**i / math.factorial(i))
        Y.append(taylor_sum)
    return Y[-1]

d = [dy,  diff(dy, var = 't y')]

solution = taylor(d, a, b, y0)
print(solution)

4.8441317887999995


**Método de Runge-Kutta**

In [122]:
def runge_kutta(f, a, b, y0, N = 10):
    h = (b - a) / N 
    t = a
    w = y0
    
    for i in range(N):
        K1 = h * f(t, w)
        K2 = h * f(t + h / 2, w + K1 / 2)
        K3 = h * f(t + h / 2, w + K2 / 2)
        K4 = h * f(t + h, w + K3)
        
        w = w + (K1 + 2 * K2 + 2 * K3 + K4) / 6
        t = t + h
    
    return w

solution = runge_kutta(dy, a, b, y0)
print(solution)


5.305363000692655


**Método de Runge-Kutta-Fehlberg**

In [123]:
def runge_kutta_fehlberg(f, a, b, y0, e = 1e-5, n = 10):
    h = (b-a) / n
    t = a
    w = y0
    
    while t < b:
        if t + h > b:
            h = b - t
        
        K1 = h * f(t, w)
        K2 = h * f(t + (1/4) * h, w + (1/4) * K1)
        K3 = h * f(t + (3/8) * h, w + (3/32) * K1 + (9/32) * K2)
        K4 = h * f(t + (12/13) * h, w + (1932/2197) * K1 - (7209/2197) * K2 + (7296/2197) * K3)
        K5 = h * f(t + h, w + (439/216) * K1 - 8 * K2 + (3680/513) * K3 - (845/4104) * K4)
        K6 = h * f(t + (1/2) * h, w - (8/27) * K1 + 2 * K2 - (3544/2565) * K3 + (1859/4104) * K4 - (11/40) * K5)
        
        R = abs( (1/360) * K1 - (128/4275) * K3 - (2197/75240) * K4 + (1/50) * K5 + (2/55) * K6 ) / h
        
        if R <= e:
            t = t + h
            w = w + 25/216 * K1 + 1408/2565 * K3 + 2197/4104 * K4 - 1/5 * K5
        
        delta = 0.84 * (e / R)**(1/4)
        h = h * min(max(delta, 0.1), 4.0)
    
    return w

solution = runge_kutta_fehlberg(dy, a, b, y0)

print(solution)


5.304781449490056
